Normalização de tamanho e divisão do dataset Ground Truth em Trainset e Testset

Com a ajuda do código a seguir, o dataset bboxManual contendo 160 fotos segmentadas manualmente normalizado para um tamanho padrão (escolhido como 100 x 100 neste projeto), e posteriormente dividido em dois blocos de forma aleatória: trainset (75% das fotos) e testset (25%). O trainset será utilizado para treinar o algoritmo de classificação, enquanto que o testset será usado para medição da performance do classificador.

In [1]:
from time import time
import pandas as pd
from skimage import io
from cv2 import resize

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


# ler metadados segmentação manual
pasta1 = "./bboxManual/"
metafile1 = "grade.csv"
filename1 = pasta1 + metafile1
df1 = pd.read_csv(filename1, sep=";")
print(df1.head(5), "\n")


# ler fotos em uma lista
fotos1 = []
for foto in df1["arquivo"]:
    fullname1 = pasta1 + foto
    img1 = io.imread(fullname1)
    fotos1.append(img1)

    
# normalizar fotos para dimensão única
d = 100
dim = (d, d)
fotos2 = [resize(img1, dim) for img1 in fotos1]


# criar arrays de predictors e outcomes
X = [f2.flatten() for f2 in fotos2]
y = df1["objeto"]

# dividir os dados em trainset e testset
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

   sequencia   objeto tipo_obj   fundo  iluminacao transformacao responsavel  \
0          3  alicate        a  branco  indoor dia   bbox manual        Ciro   
1          3    chave        a  branco  indoor dia   bbox manual        Ciro   
2          3   caneta        a  branco  indoor dia   bbox manual        Ciro   
3          3    livro        a  branco  indoor dia   bbox manual        Ciro   
4          3  caderno        a  branco  indoor dia   bbox manual        Ciro   

    arquivo  
0  1098.jpg  
1  1101.jpg  
2  1104.jpg  
3  1107.jpg  
4  1110.jpg   



Geração do Feature Vector

Uma vez realizado a normalização das imagens e criado os datasets de treinamento e testes, as imagens foram submetidas ao processo de análise de Componentes Principais (PCA). Este processo demanda a definição de cetos parâmetros, sendo o proncipal o número de components principais (n_components). A equipe decidiu trabalhar com n_components = 110, o que implica em uma redução de features de treinamento de 10000 (= 100 x 100) para 110.

Como resultado do PCA, um conjunto de autovetores é gerado. O próximo passo é então projetar as imagens e calcular suas coordenadas para este novo conjunto de autovetores.

In [2]:
# calcular PCA
n_components = 110

print("PCA - Extracting eigenvectors")
t0 = time()

pca = PCA(n_components=n_components,
          svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

h = d
w = d
eigenfaces = pca.components_.reshape((n_components, h, w))
print("PCA - Projecting to new orthonormal coordinates")
t0 = time()

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))




PCA - Extracting eigenvectors
done in 0.303s
PCA - Projecting to new orthonormal coordinates
done in 0.029s


Treinamento do modelo de classificação

Após a redução de features/predictors do set de imagens para aqueles mais relevantes, é gerado o algoritmo de classificação com o SVM:

In [3]:
# treinar o modelo com SVM
print("SVM - Fitting the classifier to the training set")
t0 = time()

param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf',
                       class_weight='balanced'),
                   param_grid)
clf = clf.fit(X_train_pca, y_train)

print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

SVM - Fitting the classifier to the training set
done in 0.911s
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.005)


Teste do modelo de classificação com o Testset

O código a seguir utiliza o modelo gerado a partir do Trainset e classifica as 25% do total de imagens identificadas como Testset. Em seguida, um relatório de desempenho (classification report/ confusion matrix) é gerado.

In [4]:
# avaliar o modelo com o testset
print("Predicting classes on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Predicting classes on the test set
done in 0.002s
              precision    recall  f1-score   support

     alicate       1.00      1.00      1.00         3
     caderno       0.20      0.33      0.25         3
      caneca       0.83      1.00      0.91         5
      caneta       0.71      0.83      0.77         6
       chave       1.00      0.67      0.80         3
      colher       1.00      0.62      0.77         8
        faca       0.50      0.50      0.50         4
       garfo       1.00      0.67      0.80         3
       livro       0.50      0.60      0.55         5

    accuracy                           0.70        40
   macro avg       0.75      0.69      0.70        40
weighted avg       0.76      0.70      0.71        40

[[3 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 2]
 [0 0 5 0 0 0 0 0 0]
 [0 0 0 5 0 0 0 0 1]
 [0 0 1 0 2 0 0 0 0]
 [0 1 0 1 0 5 1 0 0]
 [0 1 0 1 0 0 2 0 0]
 [0 0 0 0 0 0 1 2 0]
 [0 2 0 0 0 0 0 0 3]]


A Confusion Matrix acima mostrou um desempenho satisfatório na identificação das seguints classes: alicate, caneca, caneta, chave, colher, grafo. As precisões ficaram entre 0,71 e 1,00.

Merece destaque a classificação de alicates: tanto a identificação positiva (precision) quanto a identificação negativa (recall) obtiveram escores máximos.

Por outro lado, o escore balanceado "f1-score" indica classificação bastante deficiente para caderno (0,25). Facas e livros apresentaram desempenho do f1-score ao redor de 0,50. Demais itens obtiveram um escore balanceado relativamente alto (acima de 0,77).

Por fim, o "Overall Accuracy" do dataset "ground truth" ficou em 0,70.


Classificação da segmentação automática

Como última etapa do projeto, classificamos o dataset "bboxOtsu", o qual foi segmentado de forma automática.

In [5]:
###
### avaliar o modelo com o dataset "bboxOtsu"
###

# ler metadados segmentação Otsu
pasta1 = "./bboxOtsu/"
metafile1 = "grade.csv"
filename1 = pasta1 + metafile1
df1 = pd.read_csv(filename1, sep=";")
#print(df1.head(5), "\n")


# ler fotos em uma lista
fotos1 = []
for foto in df1["arquivo"]:
    fullname1 = pasta1 + foto
    img1 = io.imread(fullname1)
    fotos1.append(img1)

    
# normalizar fotos para dimensão única
d = 100
dim = (d, d)
fotos2 = [resize(img1, dim) for img1 in fotos1]


# criar arrays de predictors e outcomes
X = [f2.flatten() for f2 in fotos2]
y = df1["objeto"]


# calcular PCA
n_components = 110

print("PCA - Extracting eigenvectors")
t0 = time()

pca = PCA(n_components=n_components,
          svd_solver='randomized',
          whiten=True).fit(X)
print("done in %0.3fs" % (time() - t0))

h = d
w = d
eigenfaces = pca.components_.reshape((n_components, h, w))
print("PCA - Projecting to new orthonormal coordinates")
t0 = time()

X_pca = pca.transform(X)
print("done in %0.3fs" % (time() - t0))


# avaliar o modelo com o dataset completo bboxOtsu
print("Predicting classes on bbox Otsu")
t0 = time()
y_pred = clf.predict(X_pca)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y, y_pred))
print(confusion_matrix(y, y_pred))


PCA - Extracting eigenvectors
done in 2.490s
PCA - Projecting to new orthonormal coordinates
done in 0.131s
Predicting classes on bbox Otsu
done in 0.036s
              precision    recall  f1-score   support

     alicate       0.07      0.08      0.07       108
     caderno       0.25      0.37      0.30       108
      caneca       0.07      0.14      0.09       108
      caneta       0.10      0.04      0.05       108
       chave       0.15      0.17      0.16       108
      colher       0.04      0.06      0.05       108
        copo       0.00      0.00      0.00       108
        faca       0.08      0.13      0.10       108
       garfo       0.12      0.06      0.08       108
       livro       0.19      0.07      0.11       108

    accuracy                           0.11      1080
   macro avg       0.11      0.11      0.10      1080
weighted avg       0.11      0.11      0.10      1080

[[ 9  8 20  6  6 37  0 14  6  2]
 [15 40 13  3 11 12  1  6  2  5]
 [ 8 26 15  4  7 20 

Os resultados indicam um desempenho bastante baixo (insatisfatório) para todas as classes. Nota-se que a matriz de classificação possui elementos esparsos, não concentrados na diagonal pincipal, o que corrobora o baixo nível de acerto na classificação. O "overall accuracy" ficou em 0.13.

A equipe acredita que a principal causa raiz está no fundo escolhido para as fotos originais, em especial o as fotos com "fundo xadrez" e "fundo mickey". Por serem fundos bastante estampados, os padrões destes fundos em muitas fotos se confundiu com o objeto segmentado, o que gerou uma imagem distorcida da classe sob análise.